In [22]:
import numpy as np
from nltk.util import ngrams
from nltk.tokenize import sent_tokenize,word_tokenize
from tensorflow import keras
import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Activation, Embedding, LSTM
from keras import optimizers

/Users/girish/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
Using TensorFlow backend.


# Question 1

In [2]:
with open('jane_austen.txt','r') as f:
    data = f.read()
    f.close()
data = data.lower()

In [3]:
sent_list = sent_tokenize(data)

def train_test_split(train_split, sent_list):
    l = np.shape(sent_list)[0]
    tr_ind = int(train_split*l)
    train_data = sent_list[:tr_ind]
    test_data = sent_list[tr_ind:]
    return train_data, test_data

train_data, test_data = train_test_split(0.8, sent_list)

In [4]:
sent_words=[]
words={}
mod_sent_list=[]
for s in train_data:
    w_list=['<s>']
    temp = word_tokenize(s)
    for w in temp:
        if w.isalpha() or (('-' in w) and len(w)>1):
            w_list.append(w)
            if w not in words:
                words[w] = 1
            else:
                words[w]+=1
    w_list.append('</s>')
    sent_words.append(w_list)
    mod_sent_list.append(' '.join(w_list))

In [5]:
words['<s>']=len(train_data)
words['</s>']=len(train_data)

In [6]:
N = len(words)
uni_gram = np.zeros((N,))
bi_gram = np.zeros((N,N))
tri_gram = np.zeros((N,N,N))
# quad_gram = np.empty((N,N,N,N))
vocab = list(words.keys())
vocab_ind={}
for i,x in enumerate(words):
    vocab_ind[x] = i
c_w = np.array(list(words.values()))

In [7]:
for s in sent_words:
    for g in words.keys():
        uni_gram[vocab_ind[g]] = words[g]
    grams = list(ngrams(s,2))
    for g in grams:
        w_i = vocab_ind[g[0]]
        w_j = vocab_ind[g[1]]
        bi_gram[w_i,w_j]+=1
    grams = list(ngrams(s,3))
    for g in grams:
        w_i = vocab_ind[g[0]]
        w_j = vocab_ind[g[1]]
        w_k = vocab_ind[g[2]]
        tri_gram[w_i,w_j,w_k]+=1

In [8]:
mle_uni = np.zeros(uni_gram.shape)
mle_bi = np.zeros(bi_gram.shape)
mle_tri = np.zeros(tri_gram.shape)
mle_uni = uni_gram/sum(uni_gram)
for i in range(N):
    mle_bi[i,:] = bi_gram[i,:]/c_w[i]
    for j in range(N):
        if bi_gram[i,j] != 0:
            mle_tri[i,j,:] = tri_gram[i,j,:]/bi_gram[i,j]

In [9]:
def Generator(model_name, mle, mle_bi):
    N = len(mle_bi)
    if model_name =='unigram':
        uni_sent_gen = []
        while(True):
            ind = np.random.choice(range(N),p=mle)
            w = vocab[ind]
            uni_sent_gen.append(w)
            if w=='</s>':
                break
        return ' '.join(uni_sent_gen)
    if model_name =='bigram':
        bi_sent_gen = ['<s>']
        s_ind_i = vocab_ind['<s>']
        while(True):
            ind = np.random.choice(range(N),p=mle[s_ind_i,:])
            w = vocab[ind]
            bi_sent_gen.append(w)
            if w=='</s>':
                break
            else:
                s_ind_i = ind
        return ' '.join(bi_sent_gen)
                
    elif model_name =='trigram':
        tri_sent_gen = ['<s>']
        s_ind_j = vocab_ind['<s>']
        while(True):
            if len(tri_sent_gen) == 1:
                ind = np.random.choice(range(N),p=mle_bi[s_ind_j,:])
            else:
                ind = np.random.choice(range(N),p=mle[s_ind_i,s_ind_j,:])
            w = vocab[ind]
            tri_sent_gen.append(w)
            if w=='</s>':
                break
            else:
                s_ind_i = s_ind_j
                s_ind_j = ind
        return ' '.join(tri_sent_gen)
        

In [10]:
uni_sent_gen={}
bi_sent_gen={}
tri_sent_gen={}
for i in range(5):
    uni_sent_gen[i+1] = Generator('unigram', mle_uni, mle_bi)
    bi_sent_gen[i+1] = Generator('bigram', mle_bi,mle_bi)
    tri_sent_gen[i+1] = Generator('trigram', mle_tri, mle_bi)

In [11]:
print('Sentences generated from unigram:\n')
for i in range(5):
    print('Sentence '+str(i+1)+':\n')
    print(uni_sent_gen[i+1])
    print('\n')
print('\nSentences generated from bigram:\n')
for i in range(5):
    print('Sentence '+str(i+1)+':\n')
    print(bi_sent_gen[i+1])   
    print('\n')
print('\nSentences generated from trigram:\n')
for i in range(5):
    print('Sentence '+str(i+1)+':\n')
    print(tri_sent_gen[i+1])
    print('\n')

Sentences generated from unigram:

Sentence 1:

since to lady same the sentences unsophisticated often the that it you and either i </s>


Sentence 2:

sad her his that poor but wickham view suppose knocks have and thankful </s>


Sentence 3:

looked the and extremely but last very </s>


Sentence 4:

pleasure the excessively began humph a <s> give them to kicking see the looked and be to perverseness quiet of that his <s> a i less them publication <s> anger first you proposal <s> folding as charming without i his hall <s> he had be ornament on any hoped the the was civil anything their wentworth only of and </s>


Sentence 5:

her not character <s> it it told be it some </s>



Sentences generated from bigram:

Sentence 1:

<s> but the conversation she curtsies off the office of his ostensible reason to himself to live it would keep their dependent to the mysterious </s>


Sentence 2:

<s> they were expected the family </s>


Sentence 3:

<s> henry crawford having every young men eyes

In [12]:
test_sent_words=[]
test_words={}
test_mod_sent_list=[]
for s in test_data:
    w_list=['<s>']
    temp = word_tokenize(s)
    for w in temp:
        if w.isalpha() or (('-' in w) and len(w)>1):
            w_list.append(w)
            if w not in words:
                words[w] = 1
            else:
                words[w]+=1
    w_list.append('</s>')
    test_sent_words.append(w_list)
    test_mod_sent_list.append(' '.join(w_list))
    
test_words['<s>']=len(train_data)
test_words['</s>']=len(train_data)

In [15]:
unigram_perp = 1.0
bigram_perp = 1.0
trigram_perp = 1.0
for s in test_sent_words[:1000]:
    for w in s:
        try: 
            if mle_uni[vocab_ind[w]] == 0:
                print(mle_uni[vocab_ind[w]])
                continue
#         unigram_prob*=mle_uni[vocab_ind[w]]
            unigram_perp*=1/(mle_uni[vocab_ind[w]]**(1/N))
        except:
            continue
    grams = list(ngrams(s,2))
    for g in grams:
        try: 
            w_i = vocab_ind[g[0]]
            w_j = vocab_ind[g[1]]
            if mle_bi[w_i,w_j] == 0:
                continue
            bigram_perp*=1/(mle_bi[w_i,w_j]**(1/N))
        except:
            continue
    grams = list(ngrams(s,3))
    for g in grams:
        try:
            w_i = vocab_ind[g[0]]
            w_j = vocab_ind[g[1]]
            w_k = vocab_ind[g[2]]
            if mle_tri[w_i,w_j,w_k] == 0:
                continue
            trigram_perp*=1/(mle_tri[w_i,w_j,w_k]**(1/N))
        except:
            continue

In [16]:
print('Perplexity of unigram:',unigram_perp)
print('Perplexity of bigram:',bigram_perp)
print('Perplexity of trigram:',trigram_perp)

Perplexity of unigram: 5028.711469886763
Perplexity of bigram: 106.34052735880084
Perplexity of trigram: 4.159985713832674


It can be seen from the perplexities that the as n increase in n-gram the performance is better.

Reference:
    https://medium.com/@shivambansal36/language-modelling-text-generation-using-lstms-deep-learning-for-nlp-ed36b224b275
Collaborated with Prateek Chennuri(161100)